# 一些基础

## 1. 变量、常量

我们要重点理解的是， TensorFlow 中通过 tf.assign(ref, value) 的方式来把 value 值赋给 ref 变量。这样子，每一次循环的时候，ref 变量才不会再做定义时候的初始化操作。

In [0]:
import tensorflow as tf

### 1.1 用 Tensorflow 实现计数器

创建变量，初始化为 $0$

In [0]:
state = tf.Variable(0, name="counter")

创建一个 $OP$ ，作用是使 `state` 增加 $1$

In [0]:
one = tf.constant(1)
buf = tf.add(state, one)
update = tf.assign(state, buf)

启动图，运行 `update` $OP$

In [4]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    print(session.run(state))

    for _ in range(3):
        session.run(update)
        print(session.run(state))

0
1
2
3


### 1.2 用 Tensorflow 实现对数组求和，再计算均值

In [0]:
num_sum = tf.Variable(0.0, dtype=tf.float32)
array = tf.constant([1.0, 2.0, 3.0, 4.0])

input_data = tf.placeholder(tf.float32)

buf = tf.add(num_sum, input_data)
update = tf.assign(num_sum, buf)

In [8]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    print("num_sum = {}".format(session.run(num_sum)))
    print("array = {}".format(session.run(array)))

    for i in range(array.shape[0]):
        session.run(update, feed_dict={input_data: session.run(array[i])})
        print(session.run(num_sum))

Tensor("Placeholder_1:0", dtype=float32)
Tensor("Const_2:0", shape=(4,), dtype=float32)
num_sum = 0.0
array = [1. 2. 3. 4.]
1.0
3.0
6.0
10.0


### 1.3 只用一个变量来实现计数器

In [11]:
state = tf.Variable(0.0, tf.float32)
add_op = tf.assign(state, state + tf.constant(1.0))

with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    print(session.run(state))

    for _ in range(3):
        session.run(add_op)
        print(session.run(state))

0.0
1.0
2.0
3.0


## 2. InteractiveSession()

InteractiveSession() 主要是避免 Session 被一个变量持有

In [0]:
a = tf.constant(1.0)
b = tf.constant(2.0)
c = tf.add(a, b)

下面两种情况是等价的

In [13]:
with tf.Session():
    print(c.eval())

session = tf.InteractiveSession()
print(c.eval())
session.close()

3.0
3.0


再看一个例子

In [0]:
a = tf.constant(1.0)
b = tf.constant(2.0)
c = tf.Variable(3.0)
d = tf.add(a, b)

session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

**这样写是错误的**

```python
print(a.run())
print(b.run())
```

In [15]:
print(a.eval())
print(b.eval())

1.0
2.0


`run()` 方法主要用来

In [16]:
x = tf.Variable(1.2)
# print(x.eval()) 还没有初始化，不能这样
x.initializer.run() # x.initializer 就是一个初始化 OP，只有 OP 才可以调用 run() 方法
print(x.eval())

session.close()

1.2
